In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%precision 4

import pyjags
import numpy as np

In [24]:
sample = np.array([
        [10, 5, 12, 14, 30],
        [20, 14, 18, 22, 50],
        [20, 14, 18, 15, 25],
        [3, 2, 1, 2, 0]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                n[i, t] ~ dpois(a[i] * N[t])
            }
        }
        
        for (t in 1:T) {
            N[t]   ~ dpois(NN)
        }
        NN      ~ dunif(10,100)
        
        for (i in 1:cp) {
            b[i]   ~ dnorm(0, 10) I(0, 1)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a']

%time model = pyjags.Model(code, data=dict(n=sample, cp=cp, T=T), chains=4)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

4 5
adapting: iterations 4000 of 4000, elapsed 0:00:00, remaining 0:00:00
CPU times: user 288 ms, sys: 0 ns, total: 288 ms
Wall time: 290 ms
sampling: iterations 11620 of 80000, elapsed 0:00:01, remaining 0:00:04
sampling: iterations 19240 of 80000, elapsed 0:00:01, remaining 0:00:04
sampling: iterations 26780 of 80000, elapsed 0:00:02, remaining 0:00:04
sampling: iterations 41496 of 80000, elapsed 0:00:03, remaining 0:00:03
sampling: iterations 56492 of 80000, elapsed 0:00:04, remaining 0:00:02
sampling: iterations 71660 of 80000, elapsed 0:00:05, remaining 0:00:01
sampling: iterations 80000 of 80000, elapsed 0:00:05, remaining 0:00:00
sampling: iterations 80000 of 80000, elapsed 0:00:05, remaining 0:00:00
CPU times: user 5.24 s, sys: 0 ns, total: 5.24 s
Wall time: 5.25 s


dict_keys(['N', 'NN', 'a'])

In [64]:
def summary(samples, varname, p=95, burnin=0, thin=1):
    values = samples[varname]
    (nb, iter, chains) = values.shape
    
    for i in range(nb):
        data = values[i][burnin::thin]

        ci = np.percentile(data, [100-p, p])
        
        print('{:<8}[{}] mean = {:>12.4f} (std = {:>10.4f}), {}% credible interval [{:>5.2f} {:>5.2f}]'.format(
          varname, i, np.mean(data), np.std(data), p, *ci))

burnin = 5000
        
for varname in myvars:
    summary(samples, varname, burnin=burnin, thin=4)

NN      [0] mean =      87.6116 (std =    10.3345), 95% credible interval [70.05 99.58]
N       [0] mean =      69.3833 (std =     7.7991), 95% credible interval [57.00 82.00]
N       [1] mean =      68.1669 (std =     7.7374), 95% credible interval [56.00 81.00]
N       [2] mean =      72.6515 (std =     7.8645), 95% credible interval [60.00 86.00]
N       [3] mean =     127.8812 (std =    22.8213), 95% credible interval [97.00 168.00]
N       [4] mean =     110.3000 (std =    28.4964), 95% credible interval [71.00 152.00]
a       [0] mean =       0.2365 (std =     0.0708), 95% credible interval [ 0.13  0.35]
a       [1] mean =       0.2592 (std =     0.0467), 95% credible interval [ 0.20  0.35]
a       [2] mean =       0.4742 (std =     0.0465), 95% credible interval [ 0.39  0.53]
a       [3] mean =       0.0301 (std =     0.0096), 95% credible interval [ 0.02  0.05]
p_noise [0] mean =       0.1245 (std =     0.0781), 95% credible interval [ 0.02  0.27]


ValueError: too many values to unpack (expected 3)

## Now with some noise

In [60]:
sample = np.array([
        [10, 5, 22, 14, 130],
        [20, 14, 18, 22, 50],
        [20, 30, 18, 200, 25],
        [3, 2, 1, 7, 0]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                m[i, t]       ~ dpois(a[i] * N[t])
                noise[i, t]   ~ dbern(p_noise)
                outl[i, t]    ~ dpois(outln)
                mm[i, t, 1]  <- m[i, t]
                mm[i, t, 2]  <- m[i, t] + outl[i, t]
                
                n[i, t]       ~ dnorm(mm[i, t, choice[i, t]], tau)
                choice[i, t] <- noise[i, t] + 1
            }
        }
        
        tau    <- 1 / sigma / sigma
        sigma   ~ dunif(0.0001, 0.001)
        
        for (t in 1:T) {
            N[t]   ~ dpois(NN)
        }
        NN      ~ dunif(10,100)
        p_noise ~ dunif(0, 1)
        
        outln   ~ dunif(10,100)
        
        for (i in 1:cp) {
            b[i]   ~ dnorm(0, 10) I(0, 1)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a', 'p_noise', 'noise', 'outln', 'tau']

%time model = pyjags.Model(code, data=dict(n=sample, cp=cp, T=T), chains=4)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

4 5
adapting: iterations 2732 of 4000, elapsed 0:00:01, remaining 0:00:00
adapting: iterations 4000 of 4000, elapsed 0:00:01, remaining 0:00:00
CPU times: user 960 ms, sys: 4 ms, total: 964 ms
Wall time: 960 ms
sampling: iterations 4404 of 80000, elapsed 0:00:01, remaining 0:00:16
sampling: iterations 8984 of 80000, elapsed 0:00:02, remaining 0:00:15
sampling: iterations 11284 of 80000, elapsed 0:00:02, remaining 0:00:15
sampling: iterations 13580 of 80000, elapsed 0:00:03, remaining 0:00:14
sampling: iterations 18176 of 80000, elapsed 0:00:04, remaining 0:00:14
sampling: iterations 20460 of 80000, elapsed 0:00:04, remaining 0:00:13
sampling: iterations 25024 of 80000, elapsed 0:00:05, remaining 0:00:12
sampling: iterations 29592 of 80000, elapsed 0:00:06, remaining 0:00:11
sampling: iterations 34168 of 80000, elapsed 0:00:07, remaining 0:00:10
sampling: iterations 38752 of 80000, elapsed 0:00:08, remaining 0:00:09
sampling: iterations 43340 of 80000, elapsed 0:00:09, remaining 0:00:08

dict_keys(['a', 'tau', 'p_noise', 'outln', 'N', 'NN', 'noise'])

In [65]:
burnin = 5000
        
for varname in myvars:
    if varname != 'noise':
        summary(samples, varname, burnin=burnin, thin=4)

NN      [0] mean =      87.6116 (std =    10.3345), 95% credible interval [70.05 99.58]
N       [0] mean =      69.3833 (std =     7.7991), 95% credible interval [57.00 82.00]
N       [1] mean =      68.1669 (std =     7.7374), 95% credible interval [56.00 81.00]
N       [2] mean =      72.6515 (std =     7.8645), 95% credible interval [60.00 86.00]
N       [3] mean =     127.8812 (std =    22.8213), 95% credible interval [97.00 168.00]
N       [4] mean =     110.3000 (std =    28.4964), 95% credible interval [71.00 152.00]
a       [0] mean =       0.2365 (std =     0.0708), 95% credible interval [ 0.13  0.35]
a       [1] mean =       0.2592 (std =     0.0467), 95% credible interval [ 0.20  0.35]
a       [2] mean =       0.4742 (std =     0.0465), 95% credible interval [ 0.39  0.53]
a       [3] mean =       0.0301 (std =     0.0096), 95% credible interval [ 0.02  0.05]
p_noise [0] mean =       0.1245 (std =     0.0781), 95% credible interval [ 0.02  0.27]
outln   [0] mean =      69.611

In [62]:
samples['noise'].shape

(4, 5, 20000, 4)

In [63]:
noise = samples["noise"]
(a, b, d, e) = noise.shape

noise.mean(axis=2).mean(axis=2)

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.25],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

## Proportionated model

In [ ]:
# todo -> m[i, t] / sum(m[: , t]) modélisé comme une gamma ???

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                m[i, t]       ~ dpois(a[i] * N[t])
                noise[i, t]   ~ dbern(p_noise)
                outl[i, t]    ~ dpois(outln)
                mm[i, t, 1]  <- m[i, t]
                mm[i, t, 2]  <- m[i, t] + outl[i, t]
                
                n[i, t]       ~ dnorm(mm[i, t, choice[i, t]], tau)
                choice[i, t] <- noise[i, t] + 1
            }
        }
        
        tau    <- 1 / sigma / sigma
        sigma   ~ dunif(0.0001, 0.001)
        
        for (t in 1:T) {
            N[t]   ~ dpois(NN)
        }
        NN      ~ dunif(10,100)
        p_noise ~ dunif(0, 1)
        
        outln   ~ dunif(10,100)
        
        for (i in 1:cp) {
            b[i]   ~ dnorm(0, 10) I(0, 1)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a', 'p_noise', 'noise', 'outln', 'tau']

%time model = pyjags.Model(code, data=dict(n=sample, cp=cp, T=T), chains=4)
%time samples = model.sample(20000, vars=myvars)
samples.keys()